In [2]:
#define _FILE_SPECS "-rw-r--r-- 1 chris 8007 Mar 25 12:33 cartsim_data.py"
#define _MAGIC_NUMBER 1147484068
import numpy as np
import math, os,   sys,  time
from time import gmtime, strftime
# from logger import logger
from datetime import date, datetime, timezone
import statistics as st
import random as rn
import matplotlib.pyplot as plt
from numpy import random as rnn
 
from sklearn import metrics
import pickle

#equations
# M \ddot P = - C( X - X_0)  - D \dot X
#  P = X  + Z(Y(t))
#  Y(t)= vt

#  v=60km/hour = 16.666 m / sec
#
#  M= 1000kg
#  X_0 =  5cm
#  C spring const so 1000kg @ 10 m/sec^2 gives 5 cm ie
#    =   5 x 10e-06
#  X roughly +- 10cm
#  road 3 components period   1 sec, 2 sec, 4 sec (random amplitude)
#  sin(Y/(16.666)),  cos(Y/(16.666)), sin(Y/(2*16.66)), cos(Y/(2*16.66))
#  choose D0 to efold in 1 second ie. D/2M = 1.0 --> D=2000
#  too small is 'bad'
#  choose sampling rate @ 4 Hz
#  magnitude road = +- 5 max * sin( Y / 16.66 m)

#  10 minute samples = 600 x 4 points @ 4 Hz
#

def Zbase ( trigtype, period, K, Y):
    
    if trigtype=='sin':
        return math.sin( K * period * Y)
    if trigtype=='cos':
        return math.cos( K * period * Y)
    
def Zbaseddot ( trigtype, period, K, Y, v):
    
#     d^2/dt^ (VT)=0
    if trigtype=='sin':
        return -math.cos( K * period * Y) * v * v * (K /period) * (K/period)
    if trigtype=='cos':
        return -math.sin( K * period * Y) * v * v * (K /period) * (K/period)
    
def Xdot(Xn, Xnm1, dT):
    return (Xn - Xnm1)/dT

def Xddot(Xn, Xnm1, Xnm2, dT):
    return (Xn - 2 * Xnm1  + Xnm2)/(dT * dT)

def getXnp1(LHS, M, D, C, Xn, Xnm1, dT):
# solve for Xnp1
#  LHS = M(Xnp1 - 2 Xn + Xnm1)/delT^2 + D(Xnp1 - Xnm1)/2delT + C Xn
  
    rval = (LHS * dT * dT - (M - D * dT/2) * Xnm1  + (2 *M - C * dT * dT) * Xn  )/(M + D * dT/2) 
    
    lv = M* (rval - 2 * Xn + Xnm1)/(dT * dT) + D* (rval - Xnm1)/(2*dT) + C * Xn
    
#    print("check %f = %f" % (LHS,lv))
          
    return  rval

def getLHSval(Zddval, Ms, Vs, Cs, X0s):
# Cs * X0 =spring force
    return - Ms * Zddval + Cs * X0s

def zRoad(coeffs, v, Y, period, maxfreq):
    
    zR =0
   
    if maxfreq >= 0.5:
       zR = coeffs[0]* Zbaseddot('cos', period, 0.5, Y, v) + zR
       zR = coeffs[1]* Zbaseddot('sin', period, 0.5, Y, v) + zR

    if maxfreq >= 1.0:
       zR = coeffs[2]* Zbaseddot('cos', period, 1.0, Y, v) + zR
       zR = coeffs[3]* Zbaseddot('sin', period, 1.0, Y, v) + zR
    
    if maxfreq >=2.0:
       zR = coeffs[4]* Zbaseddot('cos', period, 2.0, Y, v) + zR
       zR = coeffs[5]* Zbaseddot('sin', period, 2.0, Y, v) + zR
       
    if maxfreq >=4.0:
       zR = coeffs[6]* Zbaseddot('cos', period, 4.0, Y, v) + zR
       zR = coeffs[7]* Zbaseddot('sin', period, 4.0, Y, v) + zR
    
    return zR

def  getRandomCoeffs(N):
    
     ampChoice=[0.01, 0.02, 0.025, 0.03, 0.035]
     coeffs=[]

     for i in range(0,N):
          rAmpl1=rn.choice(ampChoice) 
#          rAmpl1=rAmpl
          coeffs.append(rAmpl1)
          
     return coeffs
 
 

def  compute_sim(M, D0, V, C, X0, delT, period, maxfreq, coeffs, topsample):
    
    Y=0
# start with spring at rest
    Xnp1=X0
    Xn=X0
    Xnm1=X0


    springPos=[]
    timeVal=[]
    roadSurf=[]
    
     
    for i in range(0, topsample):
    
        t= i * delT
        timeVal.append(t)
#        print("                T=%8.2f" % t)
    
        Y= V * t  
    
        Zddval= zRoad(coeffs, V, Y, period, maxfreq)
    
#        print("Zddval %f" % Zddval)
    
# LHS= -M (ddot (Z(Y))) + CX0
        LHS= getLHSval(Zddval, M, V, C, X0)
        roadSurf.append(LHS)
    
        Xnp1 = getXnp1(LHS, M, D0, C, Xn, Xnm1, delT)
        springPos.append(Xnp1)
    
#        print("Xnp1 %8.3f  Xn %8.3f Xnm1 %8.3f" % (Xnp1, Xn, Xnm1))
    
        Xnm1=Xn
        Xn=Xnp1
        
    return [roadSurf, timeVal, springPos, t]

def add_sample(M, D0, V, C, X0, delT, period, maxfreq, botsample, topsample, lval,Ncoeffs, Rcoeffs, road_input_type):
 # compute discriminant
    disc= D0*D0 - 4 * M * C   
    
    [roadSurf, timeVals, springPos, tmax]= compute_sim(M, D0, V, C, X0, delT, period, maxfreq, Rcoeffs, topsample)

    yval=[]
    Xdat=[]

#    print("sample D0=%d label=%s" % (D0,lval )) 
#    print("D=%12.2f disc  %12.2f   maxfreq= %f" % (D0, disc,   maxfreq))
#    if disc < 0:
#        print("sqrt = %f" % math.sqrt(-disc))
 
# assume botsample is > 3

    if road_input_type=='vibration':
        tupleLen=8
    elif road_input_type=='surface':
        tupleLen=2
    else:
        print("unknown road_type %s" % road_type)
        tupleLen=2
 
# in this case include roadSurf = LHS as variable
    xnorm=10000 
    for i in range(botsample,topsample):
        if tupleLen==2:
# road input
           Xdat.append([roadSurf[i]/xnorm, springPos[i-2], springPos[i-1], springPos[i]])
# in vehicle vibration
        elif tupleLen==5:
           Xdat.append([springPos[i-5], springPos[i-4], springPos[i-3], springPos[i-2], springPos[i-1], springPos[i]])
#  in vehicle vibration (long)
        elif tupleLen==8:
           Xdat.append([springPos[-8], springPos[-7], springPos[-6], springPos[i-5], springPos[i-4], springPos[i-3], springPos[i-2], springPos[i-1], springPos[i]])
        else:
            print("Unsupported tupleLen %d" % tupleLen)
            return [[],[]]
        
        yval.append(lval)
 
    return [Xdat, yval]

# This main function generates data based on user's selection of parameters    
def generate_data(number_runs, experiment_type, observation_type):

  # mass    
  M=2000 
  #   5cm compression
  X0= 0.05
  # spring const    

  C= 0.6  * 10e+04
  delT=0.25
  # interesting values 500, 5000, 15000, 25000
  
  # damping
  Dvalues=[5000, 5500, 6000, 6500, 4500, 4000, 3500, 500, 600, 700, 800, 400, 300, 200]

  LABELvalues=['good','good', 'good', 'good', 'good','good','good','bad','bad','bad','bad','bad','bad','bad']

  dindexlist=[x for x in range(0,14)]

  
  # car moves at 16.66 m/s
  V= 16.66 
  period= 16.66 
  maxfreq=4.0
  
  botsample=400 
  topsample=500
  Ncoeffs=8

  X=[]
  y=[]
  ngood=0
  nbad=0

  nruns = number_runs or 500
  experiment_type= experiment_type or 'random_roads'
  # experiment_type='standard_road'

  # initialize road
  Rcoeffs=getRandomCoeffs(Ncoeffs) 

  ldindexlist=[x for x in range(0, 100* nruns)]

  for i in range(0,nruns):
      
            dindex=rn.choice(dindexlist)

            D0=Dvalues[dindex]
            labval=LABELvalues[dindex]
      
  
            if experiment_type=='random_roads':
                Rcoeffs=getRandomCoeffs(Ncoeffs)
                

  
  #  For  in vehicle vibration road_input_type='vibration'
  #  For  road input set road_input_type='surface'
            
            road_input_type = observation_type or 'surface'
            #  road_input_type='vibration'
  
            [Xdat, yval]= add_sample(M, D0, V, C, X0, delT, period, maxfreq, botsample, topsample, labval,Ncoeffs, Rcoeffs, road_input_type)
            
  # [roadsurf, Xn-2, Xn-1, Xn]
            if yval[0]=='good':
                ngood+=1
            if yval[0]=='bad':
                nbad+=1
  
            for j in range(0, len(Xdat)):
                  X.append(Xdat[j])
                  y.append(yval[j])
                  

  print("shuffling %d entries" % len(ldindexlist))
  rn.shuffle(ldindexlist)
  # sanity check
  print(ldindexlist[0:10])

  X_rn=[]
  y_rn=[]

  for i in range(0, len(X)):
      X_rn.append(X[ldindexlist[i]])
      y_rn.append(y[ldindexlist[i]])
      
      
            
  print("made %d samples with botsample %d topsample %d" % (nruns,botsample, topsample))
  print("maxfreq= %8.2f M= %8.2f V=%8.2f C=%8.2f" % (maxfreq, M, V, C))

  totalN=len(X)
  print("Total samples %d good runs %d bad runs %d" % (totalN, ngood, nbad))

  #print(Xdat)

  print (strftime("%Y-%m-%d %H:%M:%S", gmtime()))

  return X_rn, y_rn

# Analysis

The confusion matrix is a common metric to measure the performance of a classification algorithm. In this case, only "good" and "bad" are labelled (binary). So the confusion matrix can be shown as below:

|               | Predicted(No) | Predicted(Yes)  |
| ------------- |:---------------:| -----------------:|
| **Actual(No)**   | True Negatives (TN) | False Positives (FP) |
| **Actual(Yes)**  | False Negatives (FN) | True Positives (TP) |

From the matrix, we could get the Accuracy by this formula:
$$ Accuracy = \frac {TP + TN}{TP + TN + FP + FN} $$

And the Error rate would be $1-Accuracy$, same as
$$ ErrorRate = \frac {FP + FN}{TP + TN + FP + FN} $$

## Model Selection
----

Here is the summary table of all model candidates ranked by the Accuracy.

|Model|Accuracy|AUC|Recall|Prec.|F1|Kappa|MCC|TT (Sec)|
|--- |--- |--- |--- |--- |--- |--- |--- |--- |
|Extra Trees Classifier|0.9791|0.9983|0.9789|0.9806|0.9797|0.9582|0.9582|1.709|
|Quadratic Discriminant Analysis|0.9783|0.9998|0.9579|1.0000|0.9785|0.9566|0.9576|0.036|
|K Neighbors Classifier|0.9729|0.9955|0.9731|0.9742|0.9736|0.9457|0.9457|0.213|
|Random Forest Classifier|0.9727|0.9969|0.9710|0.9759|0.9735|0.9454|0.9455|7.500|
|Decision Tree Classifier|0.9477|0.9477|0.9472|0.9510|0.9491|0.8953|0.8953|0.244|
|Light Gradient Boosting Machine|0.9339|0.9893|0.9651|0.9118|0.9376|0.8674|0.8691|0.360|
|Gradient Boosting Classifier|0.8409|0.9444|0.9266|0.7973|0.8570|0.6799|0.6898|4.073|
|Ada Boost Classifier|0.6967|0.7706|0.8287|0.6648|0.7377|0.3883|0.4018|1.082|
|Naive Bayes|0.6476|0.6815|0.7716|0.6284|0.6926|0.2898|0.2983|0.032|
|Linear Discriminant Analysis|0.5494|0.5109|0.9391|0.5356|0.6820|0.0768|0.1267|0.049|
|Ridge Classifier|0.5278|0.0000|0.9967|0.5217|0.6849|0.0279|0.0915|0.031|
|Logistic Regression|0.5148|0.5109|1.0000|0.5148|0.6797|0.0003|0.0039|0.379|
|SVM - Linear Kernel|0.5147|0.0000|1.0000|0.5147|0.6796|0.0000|0.0000|0.062|


Tree models and KNN have very good accuracy compared to other models. However, most of tree classifiers need more time to compute (takes seconds). Therefore, I chose the common **KNN model** as the estimator in this case, which has a pretty good Accuracy and AUC as well as short training time (0.213s).

In [41]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

## Define Function for Model Performance

In [27]:
# Create a function to generate model result
def ml_result_report(y_test, y_pred):
  # Create Confusion Matrix with testing and prediction data
  Mc = metrics.confusion_matrix(y_test, y_pred)

  totalN = Mc[0][0] + Mc[0][1] + Mc[1][0] + Mc[1][1]
  misclassifiedN = Mc[0][1] + Mc[1][0]

  errorRate = misclassifiedN / totalN

  print("confusion matrix: on test data set")
  print(Mc)

  print("errorRate %5.3f" % errorRate)
  # print (strftime("%Y-%m-%d %H:%M:%S", gmtime()))
  print("N=%d : experiment_type: %s  road_input_type: %s" % (number_runs, experiment_type, observation_type))

  return errorRate

## Define a Model Pipeline

In [6]:
# Create a pipeline to train model and then predict
def model_pipeline(X_rn, y_rn, model):
  # Split samples
  X_train, X_test, y_train, y_test = train_test_split(X_rn, y_rn, test_size=testfraction, random_state=42)

  # Standardization of sample
  X_scaler = StandardScaler().fit(X_train)
  X_train_scaled = X_scaler.transform(X_train)
  X_test_scaled = X_scaler.transform(X_test)

  # Train model
  model.fit(X_train_scaled, y_train)

  # Print out score
  train_score = model.score(X_train_scaled, y_train)
  model_score = model.score(X_test_scaled, y_test)
  print("Train score: %5.4f" % train_score)
  print("Model score: %5.4f" % model_score)

  # Get predict data
  y_pred = model.predict(X_test_scaled)

  return y_test, y_pred

## Define Main Function to Run 4 Replications

In [25]:
# A main function to generate avg error rate
def compute_avg_err_rate(model):
  
  list_err_rate = []

  # Main function to generate required data
  for i in range(4):
    X_rn, y_rn = generate_data(number_runs, experiment_type, observation_type)

    y_test, y_pred = model_pipeline(X_rn, y_rn, knn)
    err_rate = ml_result_report(y_test, y_pred)

    list_err_rate.append(err_rate)
    print("========== %s out of 4 replications========" % (i+1))

  print("The average error rate of 4 replications is:")
  print(sum(list_err_rate)/len(list_err_rate))

# Generate Data and Return result with 3 Models

## Parameter Settings for Each Case

N runs, Experiment type, Observation type

In [53]:
#====================================================
# Here is the list of parameter variables
# Change the following variable values for each case
#----------------------------------------------------
number_runs = 200
# experiment_type = "standard_road"
experiment_type = "random_roads"

observation_type = "surface"
# observation_type = "vibration"
testfraction = 0.3
#====================================================

## KNN Model

In [40]:
# KNN
# A n_neighbors vs accuracy plotting was done separately
knn = KNeighborsClassifier(n_neighbors=3)
compute_avg_err_rate(knn)

shuffling 50000 entries
[36867, 19471, 28194, 10806, 7384, 45232, 8538, 18767, 45858, 33489]
made 500 samples with botsample 400 topsample 500
maxfreq=     4.00 M=  2000.00 V=   16.66 C=60000.00
Total samples 50000 good runs 276 bad runs 224
2021-05-22 03:34:16
Train score: 0.9919
Model score: 0.9810
confusion matrix: on test data set
[[6557  172]
 [ 113 8158]]
errorRate 0.019
N=500 : experiment_type: random_roads  road_input_type: surface
========== 1 out of 4 replications========
shuffling 50000 entries
[26961, 14535, 40783, 11357, 960, 23015, 23381, 28512, 4891, 11422]
made 500 samples with botsample 400 topsample 500
maxfreq=     4.00 M=  2000.00 V=   16.66 C=60000.00
Total samples 50000 good runs 245 bad runs 255
2021-05-22 03:34:20
Train score: 0.9917
Model score: 0.9819
confusion matrix: on test data set
[[7595  135]
 [ 136 7134]]
errorRate 0.018
N=500 : experiment_type: random_roads  road_input_type: surface
========== 2 out of 4 replications========
shuffling 50000 entries
[47

## Naive Bayes Model

In [50]:
# NB
nb = MultinomialNB()
compute_avg_err_rate(nb)

shuffling 100000 entries
[19517, 44713, 30003, 87982, 34762, 12576, 42558, 70632, 92770, 90467]
made 1000 samples with botsample 400 topsample 500
maxfreq=     4.00 M=  2000.00 V=   16.66 C=60000.00
Total samples 100000 good runs 501 bad runs 499
2021-05-22 03:43:27
Train score: 0.9937
Model score: 0.9853
confusion matrix: on test data set
[[14715   195]
 [  247 14843]]
errorRate 0.015
N=1000 : experiment_type: random_roads  road_input_type: surface
========== 1 out of 4 replications========
shuffling 100000 entries
[32176, 47261, 78775, 93805, 34278, 97122, 31867, 12869, 74862, 42504]
made 1000 samples with botsample 400 topsample 500
maxfreq=     4.00 M=  2000.00 V=   16.66 C=60000.00
Total samples 100000 good runs 466 bad runs 534
2021-05-22 03:43:35
Train score: 0.9939
Model score: 0.9858
confusion matrix: on test data set
[[15895   186]
 [  241 13678]]
errorRate 0.014
N=1000 : experiment_type: random_roads  road_input_type: surface
========== 2 out of 4 replications========
shuffl

## Neural Network Model

In [55]:
def nn_compute_avg_err_rate():

  list_err_rate = []
  # Main function to generate required data
  for i in range(4):
    X_rn, y_rn = generate_data(number_runs, experiment_type, observation_type)
    
    # Split samples
    X_train, X_test, y_train, y_test = train_test_split(X_rn, y_rn, random_state=42)

    # Standardization
    X_scaler = StandardScaler().fit(X_train)
    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)

    # Transform y values
    label_model = LabelEncoder()
    label_model.fit(y_train)

    y_train_encoded = label_model.transform(y_train)
    y_test_encoded = label_model.transform(y_test)

    # Convert to categorical data
    y_train_categorical = to_categorical(y_train_encoded)
    y_test_categorical = to_categorical(y_test_encoded)

    # Building NN model
    model = Sequential()
    model.add(Dense(units=4, activation="relu", input_dim=4))
    model.add(Dense(units=20, activation="relu"))
    model.add(Dense(units=5, activation="relu"))
    model.add(Dense(units=2, activation="softmax"))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])

    # Fitting model
    model.fit(X_train_scaled, y_train_categorical, epochs=50, shuffle=True, verbose=2)

    # Predit values
    y_pred = model.predict(X_test_scaled)
    y_pred

    # confusion_matrix(y_test_categorical.argmax(axis=1), y_pred.argmax(axis=1))
    err_rate = ml_result_report(y_test_categorical.argmax(axis=1), y_pred.argmax(axis=1))

    list_err_rate.append(err_rate)
    print("========== %s out of 4 replications========" % (i+1))

  print("The average error rate of 4 replications is:")
  print(sum(list_err_rate)/len(list_err_rate))


In [56]:
# Run MLP function
nn_compute_avg_err_rate()

shuffling 20000 entries
[17236, 10362, 12827, 18497, 5222, 15417, 10108, 201, 7670, 7508]
made 200 samples with botsample 400 topsample 500
maxfreq=     4.00 M=  2000.00 V=   16.66 C=60000.00
Total samples 20000 good runs 104 bad runs 96
2021-05-22 03:54:03
Epoch 1/50
469/469 - 1s - loss: 0.6126 - accuracy: 0.6417
Epoch 2/50
469/469 - 0s - loss: 0.4229 - accuracy: 0.7761
Epoch 3/50
469/469 - 0s - loss: 0.1577 - accuracy: 0.9540
Epoch 4/50
469/469 - 0s - loss: 0.0817 - accuracy: 0.9735
Epoch 5/50
469/469 - 0s - loss: 0.0614 - accuracy: 0.9811
Epoch 6/50
469/469 - 0s - loss: 0.0510 - accuracy: 0.9827
Epoch 7/50
469/469 - 0s - loss: 0.0434 - accuracy: 0.9853
Epoch 8/50
469/469 - 0s - loss: 0.0383 - accuracy: 0.9881
Epoch 9/50
469/469 - 0s - loss: 0.0340 - accuracy: 0.9883
Epoch 10/50
469/469 - 0s - loss: 0.0312 - accuracy: 0.9893
Epoch 11/50
469/469 - 0s - loss: 0.0291 - accuracy: 0.9899
Epoch 12/50
469/469 - 0s - loss: 0.0263 - accuracy: 0.9902
Epoch 13/50
469/469 - 0s - loss: 0.0243 - a

In [ ]:
# INSERT ML algorithm here (X_rn, y_rn) train vs test sets


#Mc=metrics.confusion_matrix(y_rn[int(testfraction*totalN):], y_pred)

#totalN=Mc[0][0] + Mc[0][1] + Mc[1][0] + Mc[1][1]
#misclassifiedN = Mc[0][1] + Mc[1][0]

#errorRate= misclassifiedN / totalN

#print("confusion matrix: on test data set")
#print(Mc)

#print("errorRate %5.3f" % errorRate)
# print (strftime("%Y-%m-%d %H:%M:%S", gmtime()))
# print("N=%d : experiment_type: %s  road_input_type: %s" % (nruns,experiment_type, road_input_type))